# Debug notebook
This notebook is only used to troubleshoot the S2 ARD workflow processor.

## S2 ARD workflow

In [1]:
import json
from utils.prepS2 import prepareS2

def process_scene(json_data):
    loaded_json = json.loads(json_data)
    #for x in loaded_json:
    #    logger.info(f"{x}: {loaded_json[x]}")
    prepareS2(**loaded_json)

## S2 ARD job queue processor

In [2]:
import os
import logging
import rediswq

level = os.getenv("LOGLEVEL", "INFO").upper()
logging.basicConfig(format="%(asctime)s %(levelname)-8s %(name)s %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=level)

host = os.getenv("REDIS_SERVICE_HOST", "redis-master")
q = rediswq.RedisWQ(name="jobS2", host=host)

logger = logging.getLogger("worker")
logger.info(f"Worker with sessionID: {q.sessionID()}")
logger.info(f"Initial queue state: empty={q.empty()}")

2020-10-26 13:13:22 INFO     worker Worker with sessionID: da13e387-89a7-4336-99f7-61ffcc8196a4
2020-10-26 13:13:22 INFO     worker Initial queue state: empty=False


In [3]:
import datetime

while not q.empty():
    item = q.lease(lease_secs=1800, block=True, timeout=600)
    if item is not None:
        itemstr = item.decode("utf=8")
        logger.info(f"Working on {itemstr}")

        start = datetime.datetime.now().replace(microsecond=0)
        process_scene(itemstr)
        q.complete(item)
        end = datetime.datetime.now().replace(microsecond=0)
        logger.info(f"Total processing time {end - start}")
    else:
        logger.info("Waiting for work")

2020-10-26 13:13:25 INFO     worker Working on {"in_scene": "S2B_MSIL2A_20200725T223009_N0214_R072_T60KWD_20200726T001511", "s3_bucket": "public-eo-data", "s3_dir": "luigi/fiji/sentinel_2/"}
2020-10-26 13:13:25 INFO     root S2B_MSIL2A_20200725T223009_N0214_R072_T60KWD_20200726T001511.SAFE S2B_MSIL2A_20200725T223009_T60KWD Starting
2020-10-26 13:13:25 INFO     root S2B_MSIL2A_20200725T223009_N0214_R072_T60KWD_20200726T001511.SAFE S2B_MSIL2A_20200725T223009_T60KWD DOWNLOADING via GCloud
2020-10-26 13:13:25 ERROR    root S2B_MSIL2A_20200725T223009_N0214_R072_T60KWD_20200726T001511.SAFE S2B_MSIL2A_20200725T223009_T60KWD UNAVAILABLE via GCloud, try ESA
Traceback (most recent call last):
  File "/opt/notebooks/utils/prepS2.py", line 455, in prepareS2
    download_s2_granule_gcloud(in_scene, inter_dir, down_dir)
  File "/opt/notebooks/utils/prepS2.py", line 42, in download_s2_granule_gcloud
    client = storage.Client.from_service_account_json(tmpjson)
  File "/opt/conda/lib/python3.7/site-p

In [4]:
logger.info("Queue empty!")

2020-10-26 13:20:12 INFO     worker Queue empty!


---